In [49]:
# !pip install numpy pyarrow fastparquet scipy scikit-learn mlflow jupyter seaborn hyperopt xgboost

In [50]:
import numpy, scipy, pandas as pd, os, pickle, mlflow
from sklearn.feature_extraction import DictVectorizer
from sklearn.linear_model import LinearRegression
import seaborn as sns
import matplotlib.pyplot as plt

from sklearn.linear_model import Lasso
from sklearn.linear_model import Ridge

from sklearn.metrics import mean_squared_error

In [51]:
# JAN 
# https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2023-01.parquet
# https://d37ci6vzurychx.cloudfront.net/trip-data/green_tripdata_2023-01.parquet

# FEB
# https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2023-02.parquet

# MAR
# https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2023-03.parquet

In [52]:
mlflow.set_tracking_uri("sqlite:///mlflow.db")
mlflow.set_experiment("2_exp_hw2")

2024/05/29 14:02:45 INFO mlflow.tracking.fluent: Experiment with name '2_exp_hw2' does not exist. Creating a new experiment.


<Experiment: artifact_location='/workspaces/codespaces-blank/2_chap/mlruns/2', creation_time=1716991365548, experiment_id='2', last_update_time=1716991365548, lifecycle_stage='active', name='2_exp_hw2', tags={}>

In [53]:
def read_dataframe(filename):
    df = pd.read_parquet(filename)

    df.lpep_dropoff_datetime = pd.to_datetime(df.lpep_dropoff_datetime)
    df.lpep_pickup_datetime = pd.to_datetime(df.lpep_pickup_datetime)

    df['duration'] = df.lpep_dropoff_datetime - df.lpep_pickup_datetime
    df.duration = df.duration.apply(lambda td: td.total_seconds() / 60)

    df = df[(df.duration >= 1) & (df.duration <= 60)]

    categorical = ['PULocationID', 'DOLocationID']
    df[categorical] = df[categorical].astype(str)
    
    return df

In [54]:
# df_jan = pd.read_parquet('/workspaces/codespaces-blank/data/2/green_tripdata_2023-01.parquet')
# df_train = pd.read_parquet('/workspaces/codespaces-blank/data/2/green_tripdata_2023-01.parquet')
# df_feb = pd.read_parquet('/workspaces/codespaces-blank/data/2/green_tripdata_2023-02.parquet')
# df_val = pd.read_parquet('/workspaces/codespaces-blank/data/2/green_tripdata_2023-02.parquet')
# df_mar = pd.read_parquet('/workspaces/codespaces-blank/data/2/green_tripdata_2023-03.parquet')

In [55]:
df_train = read_dataframe('/workspaces/codespaces-blank/data/2/green_tripdata_2023-01.parquet')
df_val = read_dataframe('/workspaces/codespaces-blank/data/2/green_tripdata_2023-02.parquet')

In [56]:
len(df_train), len(df_val)

(65946, 62574)

In [57]:
df_train['PU_DO'] = df_train['PULocationID'] + '_' + df_train['DOLocationID']
df_val['PU_DO'] = df_val['PULocationID'] + '_' + df_val['DOLocationID']

In [58]:


categorical = ['PU_DO'] #'PULocationID', 'DOLocationID']
numerical = ['trip_distance']

dv = DictVectorizer()

train_dicts = df_train[categorical + numerical].to_dict(orient='records')
X_train = dv.fit_transform(train_dicts)

val_dicts = df_val[categorical + numerical].to_dict(orient='records')
X_val = dv.transform(val_dicts)

target = 'duration'
y_train = df_train[target].values
y_val = df_val[target].values


lr = LinearRegression()
lr.fit(X_train, y_train)

y_pred = lr.predict(X_val)

mean_squared_error(y_val, y_pred, squared=False)

/workspaces/codespaces-blank/.venv/lib/python3.10/site-packages/sklearn/metrics/_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(


6.037803764245777

In [59]:
with open('models/lin_reg.bin', 'wb') as f_out:
    pickle.dump((dv, lr), f_out)

In [60]:
with mlflow.start_run():

    mlflow.set_tag("author", "Objective_Haslett")

    mlflow.log_param("train-data-path", "./data/green_tripdata_2021-01.csv")
    mlflow.log_param("valid-data-path", "./data/green_tripdata_2021-02.csv")

    alpha = 0.1
    mlflow.log_param("alpha", alpha)
    lr = Lasso(alpha)
    lr.fit(X_train, y_train)

    y_pred = lr.predict(X_val)
    rmse = mean_squared_error(y_val, y_pred, squared=False)
    mlflow.log_metric("rmse", rmse)

    mlflow.log_artifact(local_path="models/lin_reg.bin", artifact_path="models_pickle")

/workspaces/codespaces-blank/.venv/lib/python3.10/site-packages/sklearn/metrics/_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(


In [61]:
import xgboost as xgb

from hyperopt import fmin, tpe, hp, STATUS_OK, Trials
from hyperopt.pyll import scope

train = xgb.DMatrix(X_train, label=y_train)
valid = xgb.DMatrix(X_val, label=y_val)

def objective(params):
    with mlflow.start_run():
        mlflow.set_tag("model", "xgboost")
        mlflow.log_params(params)
        booster = xgb.train(
            params=params,
            dtrain=train,
            num_boost_round=1000,
            evals=[(valid, 'validation')],
            early_stopping_rounds=50
        )
        y_pred = booster.predict(valid)
        rmse = mean_squared_error(y_val, y_pred, squared=False)
        mlflow.log_metric("rmse", rmse)

    return {'loss': rmse, 'status': STATUS_OK}

In [64]:
search_space = {
    'max_depth': scope.int(hp.quniform('max_depth', 4, 100, 1)),
    'learning_rate': hp.loguniform('learning_rate', -3, 0),
    'reg_alpha': hp.loguniform('reg_alpha', -5, -1),
    'reg_lambda': hp.loguniform('reg_lambda', -6, -1),
    'min_child_weight': hp.loguniform('min_child_weight', -1, 3),
    'objective': 'reg:linear',
    'seed': 42
}

best_result = fmin(
    fn=objective,
    space=search_space,
    algo=tpe.suggest,
    max_evals=10,
    trials=Trials()
)

  0%|          | 0/10 [00:00<?, ?trial/s, best loss=?]

/workspaces/codespaces-blank/.venv/lib/python3.10/site-packages/xgboost/core.py:160: UserWarning: [14:12:27] WARNING: /workspace/src/objective/regression_obj.cu:209: reg:linear is now deprecated in favor of reg:squarederror.
  warnings.warn(smsg, UserWarning)



[0]	validation-rmse:8.92025                           
[1]	validation-rmse:8.55564                           
[2]	validation-rmse:8.22079                           
[3]	validation-rmse:7.91953                           
[4]	validation-rmse:7.64425                           
[5]	validation-rmse:7.39950                           
[6]	validation-rmse:7.17153                           
[7]	validation-rmse:6.97193                           
[8]	validation-rmse:6.78911                           
[9]	validation-rmse:6.62526                           
[10]	validation-rmse:6.48092                          
[11]	validation-rmse:6.34868                          
[12]	validation-rmse:6.22937                          
[13]	validation-rmse:6.12538                          
[14]	validation-rmse:6.03194                          
[15]	validation-rmse:5.95086                          
[16]	validation-rmse:5.86930                          
[17]	validation-rmse:5.80594                          
[18]	valid

/workspaces/codespaces-blank/.venv/lib/python3.10/site-packages/sklearn/metrics/_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(

/workspaces/codespaces-blank/.venv/lib/python3.10/site-packages/xgboost/core.py:160: UserWarning: [14:14:26] WARNING: /workspace/src/objective/regression_obj.cu:209: reg:linear is now deprecated in favor of reg:squarederror.
  warnings.warn(smsg, UserWarning)



[0]	validation-rmse:5.60661                                                     
[1]	validation-rmse:5.28567                                                     
[2]	validation-rmse:5.23631                                                     
[3]	validation-rmse:5.22136                                                     
[4]	validation-rmse:5.21579                                                     
[5]	validation-rmse:5.20950                                                     
[6]	validation-rmse:5.19499                                                     
[7]	validation-rmse:5.19181                                                     
[8]	validation-rmse:5.19303                                                     
[9]	validation-rmse:5.18383                                                     
[10]	validation-rmse:5.18372                                                    
[11]	validation-rmse:5.18208                                                    
[12]	validation-rmse:5.18001

...

/workspaces/codespaces-blank/.venv/lib/python3.10/site-packages/sklearn/metrics/_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(

/workspaces/codespaces-blank/.venv/lib/python3.10/site-packages/xgboost/core.py:160: UserWarning: [14:20:51] WARNING: /workspace/src/objective/regression_obj.cu:209: reg:linear is now deprecated in favor of reg:squarederror.
  warnings.warn(smsg, UserWarning)



[0]	validation-rmse:5.66680                                                    
[1]	validation-rmse:5.37885                                                    
[2]	validation-rmse:5.35160                                                    
[3]	validation-rmse:5.33798                                                    
[4]	validation-rmse:5.33580                                                    
[5]	validation-rmse:5.32860                                                    
[6]	validation-rmse:5.32860                                                    
[7]	validation-rmse:5.32309                                                    
[8]	validation-rmse:5.31662                                                    
[9]	validation-rmse:5.31321                                                    
[10]	validation-rmse:5.31414                                                   
[11]	validation-rmse:5.31239                                                   
[12]	validation-rmse:5.30646            

/workspaces/codespaces-blank/.venv/lib/python3.10/site-packages/sklearn/metrics/_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(

/workspaces/codespaces-blank/.venv/lib/python3.10/site-packages/xgboost/core.py:160: UserWarning: [14:21:07] WARNING: /workspace/src/objective/regression_obj.cu:209: reg:linear is now deprecated in favor of reg:squarederror.
  warnings.warn(smsg, UserWarning)



[0]	validation-rmse:7.85501                                                    
[1]	validation-rmse:6.87988                                                    
[2]	validation-rmse:6.25347                                                    
[3]	validation-rmse:5.86580                                                    
[4]	validation-rmse:5.63118                                                    
[5]	validation-rmse:5.48775                                                    
[6]	validation-rmse:5.39326                                                    
[7]	validation-rmse:5.33464                                                    
[8]	validation-rmse:5.29713                                                    
[9]	validation-rmse:5.27564                                                    
[10]	validation-rmse:5.25642                                                   
[11]	validation-rmse:5.24229                                                   
[12]	validation-rmse:5.23102            

/workspaces/codespaces-blank/.venv/lib/python3.10/site-packages/sklearn/metrics/_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(



In [65]:
mlflow.xgboost.autolog(disable=True)

with mlflow.start_run():
    
    train = xgb.DMatrix(X_train, label=y_train)
    valid = xgb.DMatrix(X_val, label=y_val)

    best_params = {
        'learning_rate': 0.09585355369315604,
        'max_depth': 30,
        'min_child_weight': 1.060597050922164,
        'objective': 'reg:linear',
        'reg_alpha': 0.018060244040060163,
        'reg_lambda': 0.011658731377413597,
        'seed': 42
    }

    mlflow.log_params(best_params)

    booster = xgb.train(
        params=best_params,
        dtrain=train,
        num_boost_round=1000,
        evals=[(valid, 'validation')],
        early_stopping_rounds=10
    )

    y_pred = booster.predict(valid)
    rmse = mean_squared_error(y_val, y_pred, squared=False)
    mlflow.log_metric("rmse", rmse)

    with open("models/preprocessor.b", "wb") as f_out:
        pickle.dump(dv, f_out)
    mlflow.log_artifact("models/preprocessor.b", artifact_path="preprocessor")

    mlflow.xgboost.log_model(booster, artifact_path="models_mlflow")

/workspaces/codespaces-blank/.venv/lib/python3.10/site-packages/xgboost/core.py:160: UserWarning: [14:43:38] WARNING: /workspace/src/objective/regression_obj.cu:209: reg:linear is now deprecated in favor of reg:squarederror.
  warnings.warn(smsg, UserWarning)


[0]	validation-rmse:8.73788
[1]	validation-rmse:8.22960
[2]	validation-rmse:7.78914
[3]	validation-rmse:7.40823
[4]	validation-rmse:7.08398
[5]	validation-rmse:6.80130
[6]	validation-rmse:6.56559
[7]	validation-rmse:6.35942
[8]	validation-rmse:6.18716
[9]	validation-rmse:6.04364
[10]	validation-rmse:5.91994
[11]	validation-rmse:5.81441
[12]	validation-rmse:5.72701
[13]	validation-rmse:5.65236
[14]	validation-rmse:5.58821
[15]	validation-rmse:5.53629
[16]	validation-rmse:5.49451
[17]	validation-rmse:5.45443
[18]	validation-rmse:5.42147
[19]	validation-rmse:5.39347
[20]	validation-rmse:5.37267
[21]	validation-rmse:5.35128
[22]	validation-rmse:5.33257
[23]	validation-rmse:5.31780
[24]	validation-rmse:5.30449
[25]	validation-rmse:5.29542
[26]	validation-rmse:5.28365
[27]	validation-rmse:5.27505
[28]	validation-rmse:5.26725
[29]	validation-rmse:5.26004
[30]	validation-rmse:5.25388
[31]	validation-rmse:5.24812
[32]	validation-rmse:5.24406
[33]	validation-rmse:5.24068
[34]	validation-rmse:5.2

/workspaces/codespaces-blank/.venv/lib/python3.10/site-packages/sklearn/metrics/_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(
/workspaces/codespaces-blank/.venv/lib/python3.10/site-packages/xgboost/core.py:160: UserWarning: [14:44:46] WARNING: /workspace/src/c_api/c_api.cc:1240: Saving into deprecated binary model format, please consider using `json` or `ubj`. Model format will default to JSON in XGBoost 2.2 if not specified.
  warnings.warn(smsg, UserWarning)


In [67]:
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor, ExtraTreesRegressor
from sklearn.svm import LinearSVR

mlflow.sklearn.autolog()

for model_class in (RandomForestRegressor, GradientBoostingRegressor, ExtraTreesRegressor, LinearSVR):

    with mlflow.start_run():

        mlflow.log_param("train-data-path", "./data/green_tripdata_2021-01.csv")
        mlflow.log_param("valid-data-path", "./data/green_tripdata_2021-02.csv")
        mlflow.log_artifact("models/preprocessor.b", artifact_path="preprocessor")

        mlmodel = model_class()
        mlmodel.fit(X_train, y_train)

        y_pred = mlmodel.predict(X_val)
        rmse = mean_squared_error(y_val, y_pred, squared=False)
        mlflow.log_metric("rmse", rmse)
        

2024/05/29 14:49:14 WARNING mlflow.utils.autologging_utils: You are using an unsupported version of sklearn. If you encounter errors during autologging, try upgrading / downgrading sklearn to a supported version, or try upgrading MLflow.
2024/05/29 14:49:14 WARNING mlflow.utils.autologging_utils: MLflow autologging encountered a warning: "/workspaces/codespaces-blank/.venv/lib/python3.10/site-packages/_distutils_hack/__init__.py:18: UserWarning: Distutils was imported before Setuptools, but importing Setuptools also replaces the `distutils` module in `sys.modules`. This may lead to undesirable behaviors or errors. To avoid these issues, avoid using distutils directly, ensure that setuptools is installed in the traditional way (e.g. not an editable install), and/or make sure that setuptools is always imported before distutils."
2024/05/29 14:49:14 WARNING mlflow.utils.autologging_utils: MLflow autologging encountered a warning: "/workspaces/codespaces-blank/.venv/lib/python3.10/site-pac

AssertionError: /home/codespace/.python/current/lib/python3.10/distutils/core.py